In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, time
import requests
import logging
import json
import numpy as np
import time
import pandas as pd
from utils import get_config
import marshaltools
from tns_bulk_report import addBulkReport, getBulkReportReply, TNSClient
#import prepare_report as pr

/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# These are the selection parameters, so sync with T3 later
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
minrefframes = 1    # Cut away alert information where this is not positive (how can it not be, but ok...)
apikey = "a3f9bcbbe6a26a1ae97a0eeefe465932e68cba83" # public alerts
#apikey = "5c8fe85084b1c6f281f5d0e135e166af361ecfc3" # private partnership data

In [17]:
config = get_config()
username = config['username']
date = '2018-09-21'
logpath = './logs/'
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'tnssubmit_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [18]:
# Temporary reread detections
slackfilepath = config['slackfilepath']
print(slackfilepath)
df_sum = pd.read_csv(os.path.join(slackfilepath, "Summary_%s.csv"%(date)))
df_phot = pd.read_csv(os.path.join(slackfilepath, "Photometry_%s.csv"%(date)))

if minrefframes>0:
    df_phot = df_phot[df_phot["nframesref"]>minrefframes]

# Examine new candidates, build a list of the onces fulfilling basic criteria
sncandidates = []
for sne in df_sum.iterrows():
    print( sne[1]["ztf_name"] )
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    if np.min(magpsf)> minpeakmag:
        msg = "%s: Skipping %s, never reached limit %s"%(date,sne[1]["ztf_name"],minpeakmag)
        print(msg)
        continue
    if len(magpsf)< mindet:
        msg = "%s: Skipping %s, less than %s detections"%(date,sne[1]["ztf_name"],mindet)
        print(msg)
        continue
    sncandidates.append(sne[1]["ztf_name"])
    continue
        
    

print("Left with %s candidates"%(len(sncandidates)))
logger.info("Found %s candidates to submit"%(len(sncandidates)))

/home/jnordin/Downloads/
ZTF18absldfl
ZTF18aaqiivp
ZTF18abqlpgq
ZTF18abgmcmv
ZTF18aayaphx
ZTF18abnvotk
ZTF18abrlupd
ZTF18abeajml
ZTF18abqboud
ZTF18aavqdyq
ZTF18abobkii
ZTF18abtkpxi
2018-09-21: Skipping ZTF18abtkpxi, never reached limit 19.5
ZTF18abhpdpk
2018-09-21: Skipping ZTF18abhpdpk, never reached limit 19.5
ZTF18abehrdw
2018-09-21: Skipping ZTF18abehrdw, less than 5 detections
ZTF18absmsbm
2018-09-21: Skipping ZTF18absmsbm, never reached limit 19.5
ZTF17aaaafcd
ZTF18abjrbza
2018-09-21: Skipping ZTF18abjrbza, less than 5 detections
ZTF18ablrlbm
ZTF18abtgzie
2018-09-21: Skipping ZTF18abtgzie, never reached limit 19.5
ZTF18abtghbi
ZTF18abtnflo
ZTF18aaupjtd
ZTF18abpamut
ZTF18abmwfxd
ZTF18abtcdfv
2018-09-21: Skipping ZTF18abtcdfv, never reached limit 19.5
ZTF18abryewg
ZTF18abodiov
2018-09-21: Skipping ZTF18abodiov, never reached limit 19.5
ZTF18abnvoel
ZTF18absrcps
ZTF18absdgon
ZTF18abqdtes
ZTF18abfhryc
ZTF18abaaeip
2018-09-21: Skipping ZTF18abaaeip, never reached limit 19.5
ZTF18absbg

2018-09-21: Skipping ZTF18abukiix, never reached limit 19.5
ZTF18absoieo
2018-09-21: Skipping ZTF18absoieo, less than 5 detections
ZTF18abuksxd
ZTF18aburrut
2018-09-21: Skipping ZTF18aburrut, never reached limit 19.5
ZTF18absljlb
2018-09-21: Skipping ZTF18absljlb, never reached limit 19.5
ZTF18abvuaiv
2018-09-21: Skipping ZTF18abvuaiv, never reached limit 19.5
ZTF18abvnwzz
ZTF18abuehkg
2018-09-21: Skipping ZTF18abuehkg, less than 5 detections
ZTF18abvbpku
2018-09-21: Skipping ZTF18abvbpku, less than 5 detections
ZTF18abtmixs
2018-09-21: Skipping ZTF18abtmixs, never reached limit 19.5
ZTF18abvefxs
2018-09-21: Skipping ZTF18abvefxs, less than 5 detections
ZTF18abttaxw
ZTF18abuyuqy
2018-09-21: Skipping ZTF18abuyuqy, never reached limit 19.5
ZTF18abwdhgw
ZTF18abwgtih
2018-09-21: Skipping ZTF18abwgtih, less than 5 detections
ZTF18aakgsmi
ZTF18aabilqu
ZTF18abvmqnk
ZTF18abwkrbl
2018-09-21: Skipping ZTF18abwkrbl, less than 5 detections
ZTF18abxfdcb
2018-09-21: Skipping ZTF18abxfdcb, never reac

INFO:jnordin:Found 217 candidates to submit


2018-09-21: Skipping ZTF18abyctym, never reached limit 19.5
ZTF18abecggb
2018-09-21: Skipping ZTF18abecggb, less than 5 detections
ZTF18abwdpla
2018-09-21: Skipping ZTF18abwdpla, less than 5 detections
ZTF18abxgxpa
2018-09-21: Skipping ZTF18abxgxpa, never reached limit 19.5
ZTF18abyanoj
2018-09-21: Skipping ZTF18abyanoj, never reached limit 19.5
ZTF18abvfbnp
2018-09-21: Skipping ZTF18abvfbnp, never reached limit 19.5
ZTF18aaabltn
2018-09-21: Skipping ZTF18aaabltn, less than 5 detections
ZTF18aaghxan
2018-09-21: Skipping ZTF18aaghxan, less than 5 detections
ZTF18abxstkt
2018-09-21: Skipping ZTF18abxstkt, never reached limit 19.5
ZTF18aaubaen
2018-09-21: Skipping ZTF18aaubaen, less than 5 detections
ZTF18aavzlyn
2018-09-21: Skipping ZTF18aavzlyn, less than 5 detections
ZTF18abkwyix
2018-09-21: Skipping ZTF18abkwyix, never reached limit 19.5
ZTF18abxbwwf
2018-09-21: Skipping ZTF18abxbwwf, less than 5 detections
ZTF18abwwuug
2018-09-21: Skipping ZTF18abwwuug, less than 5 detections
ZTF18ab

In [5]:
# Use Marshal data for now
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False,
                    sfd_dir='/home/jnordin/local/git/simsurvey-examples/data/sfddata-master')
cpl = marshaltools.ProgramList('Cosmology',load_candidates=False,
                    sfd_dir='/home/jnordin/local/git/simsurvey-examples/data/sfddata-master')

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 223 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Initialized ProgramList for program Cosmology (ID 2)
INFO:marshaltools.ProgramList:Loaded 480 saved sources for program Cosmology.


In [13]:
# What fraction of the above candidates are in these programs
inprog = {'both':[],'cosm':[], 'ampel':[],'none':[]}
for sn in sncandidates:
    if sn in cpl.sources and sn in pl.sources:
        inprog['both'].append(sn)
    elif sn in cpl.sources:
        inprog['cosm'].append(sn)
    elif sn in pl.sources:
        inprog['ampel'].append(sn)
    else:
        inprog['none'].append(sn)

for source, count in inprog.items():
    print('%s count %s'%(source,len(count)))
# Ok, have to use both programs...

both count 15
cosm count 23
ampel count 18
none count 56


In [19]:
# TNS base entities
atfields = ["obs_date", "ra", "dec", "diffmaglim", "programid", "candid", "magpsf", "sigmapsf", "filter_id"]
atpos = {field:k for k,field in enumerate(atfields)}
# Instrument/survey values
ztf_tns = {"flux_units":"1", "instrument_value":196,"exptime":30,"Observer":"ZTF"}
# Have to tie to TNS filter numbering
# ZTFg 110 ZTFr 111 ZTFi 112
tns_filters = {'g':110,'r':111,'i':112}
exptime = 30

tnsclient = TNSClient(logger,True,{'api_key':apikey})

In [20]:
# Start collecting the data into dictionaries for submission
atreport = {}
atcounter = 0
atreportlists = []   # These cannot have more than one hundred entries, so we do this cumbersome way
tnsid = {}
skipuntil = None
for sn in sncandidates:
    print(sn)
    if skipuntil is not None and not skipuntil==sn:
        continue
    skipuntil = None

   
    

    # Start defining AT dict
    atdict = {"groupid":48, "reporter":"Automatic AMPEL report on behalf of the Zwicky Transient Facility", 
              "at_type": 1, "internal_name":sn, 
              "internal_name_format":{"postfix": "","prefix": "ZTF", "year_format": "YY"}
             }
        

    
    # Hmm, we do not have upper limits. Load from marshal for now?
    if sn in cpl.sources:
        lc = cpl.get_lightcurve(sn)
        del cpl.lightcurves[sn]
    elif sn in pl.sources:
        lc = pl.get_lightcurve(sn)
        del pl.lightcurves[sn]
    else:
        msg = "%s not in marshal, skipping from trial tns"%(sn)
        print(msg)
        logger.info(msg)
        continue

        
    # Check if already at TNS
    [exist,tnsname] = tnsclient.checkTNS(lc.ra,lc.dec)
    
    if exist is True:
        msg = ("%s already at TNS under name %s"%(sn,tnsname))
        logger.info(msg)
        print(msg)
        tnsid[sn] = tnsname
        continue
    elif exist is False:
        print("Try TNS submit")
    else:
        sys.exit("Something went wrong with the TNS submit %s"%(tnsname))
        
 
        
        
        
    # Cut down to P48 g and r (note that we right now could include high density fields)
    t = lc.table[(lc.table["instrument"]=='P48+ZTF')]
    t = t[((t['filter']=='g') | (t['filter']=='r') )]
        
    # Get date and info on upper limit
    firstdet_jd = t[(t["magpsf"]<30)]['jdobs'].min()
    if firstdet_jd == t['jdobs'].min():
        msg = ("Skipping. %s detected at first alert time %s, no previous upper limit"%(sn,firstdet_jd))
        print( msg )
        logger.info(msg)
        continue
    
    
    atdict["discovery_datetime"] = firstdet_jd
    print(firstdet_jd)
    nondet_jd = t[(t["magpsf"]==99) & (t['jdobs']<firstdet_jd) ]['jdobs'].max()
    msg = "First det at %s w nondetect at %s"%(firstdet_jd,nondet_jd)
    print(msg)
    
    f = t[t['jdobs']==nondet_jd]
    atdict["non_detection"] = {"obsdate": nondet_jd,
                              "limiting_flux": f["limmag"][0],
                              "filter_value": tns_filters[f["filter"][0]]}
    atdict["non_detection"].update(ztf_tns)  # Add the default ZTF values
          
          
    # Add position
    atdict["ra"] = {"value": lc.ra, "error" : 0.2, "units":"arcsec"}
    atdict["dec"] = {"value": lc.dec, "error" : 0.2, "units":"arcsec"}

          
          
          
    # Photometry
    # Find the first two detections and create photometry atdict
    
    atdict["photometry"] = {"photometry_group":{}}
    f = t[t['magpsf']<30]  # Non upper limits
    print(f)
    for counter, i in enumerate(f['jdobs'].argsort()[:2]):
        print(f["jdobs"][i])
        photdict = {"obsdate":f["jdobs"][i],
                    "flux":f["magpsf"][i],
                    "flux_err":f["sigmamagpsf"][i],
                    "limiting_flux":f["limmag"][i],
                    "filter_value":tns_filters[f["filter"][i]]
                    }
        photdict.update(ztf_tns)  # Add the default ZTF values
        atdict["photometry"]["photometry_group"][counter] = photdict
    
    # Proprietery
    atdict["proprietary_period"]: {
        "proprietary_period_units": "days", 
        "proprietary_period_value": "0"}
    atdict["proprietary_period_groups"]: [ 48 ]
    
        

    # Here we should collect files to upload and add these to the AT dict
        
    atreport[str(atcounter)] = atdict
    atcounter += 1
    logger.info('Added %s as id %s to TNS list'%(sn,atcounter))


    
    if atcounter==50:  # Change to 99
        atreportlists.append( atreport )
        atreport = {}
        atcounter = 0
        
    
# Add the last dict
atreportlists.append( {"at_report":atreport} )

ZTF18absldfl


INFO:jnordin:ZTF18absldfl already at TNS under name SN2018fsm
INFO:jnordin:ZTF18aaqiivp not in marshal, skipping from trial tns


ZTF18absldfl already at TNS under name SN2018fsm
ZTF18aaqiivp
ZTF18aaqiivp not in marshal, skipping from trial tns
ZTF18abqlpgq


INFO:jnordin:ZTF18abqlpgq already at TNS under name SN2018fsf


ZTF18abqlpgq already at TNS under name SN2018fsf
ZTF18abgmcmv


...

ZTF18abulkrl already at TNS under name SN2018ghc
ZTF18abvxbfh
ZTF18abvxbfh not in marshal, skipping from trial tns
ZTF18abvkhul


INFO:jnordin:ZTF18abvkhul already at TNS under name AT2018hnp
INFO:jnordin:ZTF18abtlwoj not in marshal, skipping from trial tns


ZTF18abvkhul already at TNS under name AT2018hnp
ZTF18abtlwoj
ZTF18abtlwoj not in marshal, skipping from trial tns
ZTF18abuiknd


INFO:jnordin:ZTF18abuiknd already at TNS under name SN2018gtg
INFO:jnordin:ZTF18absgmfy not in marshal, skipping from trial tns
INFO:jnordin:ZTF18abvyvlr not in marshal, skipping from trial tns


ZTF18abuiknd already at TNS under name SN2018gtg
ZTF18absgmfy
ZTF18absgmfy not in marshal, skipping from trial tns
ZTF18abvyvlr
ZTF18abvyvlr not in marshal, skipping from trial tns


In [21]:
atreportlists

[{'at_report': {}}]

In [10]:
for atreport in atreportlists:
    if len(atreport['at_report'])==0 : continue
        
    # try to submit this
    reportid = addBulkReport(atreport, apikey, logger)
    if reportid:
        print('TNS report ID %s'%(reportid) )
        response3 = getBulkReportReply(reportid, apikey, logger)
        if response3 is None:
            print('Did TNS not have enough time to process?')
            time.sleep(5)
            response3 = getBulkReportReply(reportid, apikey, logger)
            if response3 is None:
                print('Really failed reply reading! ')
                break
        for r in response3:
            print(r)
    else:
        print('Send failure - why? Try again?')
